In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import os
from urllib.request import urlretrieve
from multiprocessing import Pool
from functools import partial

In [ ]:
def download_file(uri):
    url = uri[0]; out_dir = uri[1]
    fname = url.rsplit("/",1)[-1]
    print(f'Started downloading {fname}\n')
    urlretrieve(url, os.path.join(out_dir, fname))
    print(f'Completed downloading {fname}')
    return fname

In [ ]:
PATH_OUT = os.path.abspath(os.getcwd())

In [ ]:
langlist = ['en', 'ru', 'ja', 'de', 'fr', 'it', 'pl', 'fa']
pool = Pool(processes=10)
for lang in langlist:
    base_url = f'https://archive.org/download/{lang}wiki-20210401'
    print(base_url)
    html_text = requests.get(base_url).text
    soup = BeautifulSoup(html_text, 'html.parser')

    pattern = re.compile(re.escape(lang)+r'wiki-20210401-pages-articles\d{1,2}.xml.*.bz2')

    out_dir = os.path.join(PATH_OUT, f'{lang}_20210401')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    toDownload = []
    for row in soup.findAll('tr'):
        for elem in row.findAll('td'):
            if elem.find('a') is not None:
                wiki_url = elem.a['href']
                if re.match(pattern, wiki_url):
                    toDownload.append([os.path.join(base_url,wiki_url), out_dir])
    if len(toDownload) == 0:
        toDownload.append([os.path.join(base_url,f'{lang}wiki-20210401-pages-articles.xml.bz2'), out_dir])

    return_msg_list = pool.map(download_file, toDownload)
    
    print(f'Completed downloading {return_msg_list} for {lang}wiki')